In [2]:
from google.colab import drive
drive.mount('/content/drive')


import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import ndimage
import tensorflow as tf
import math
import cv2
import random
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, experimental, GlobalAveragePooling3D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Lambda
from tensorflow.keras.layers import Conv3D, MaxPooling3D, ZeroPadding3D
from tensorflow.keras.layers import Conv3D, MaxPooling3D, BatchNormalization,  Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l1,l2
from tensorflow.keras.layers import LeakyReLU
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import callbacks

import keras
import os
import numpy as np



Mounted at /content/drive


In [5]:
!git clone https://github.com/Nibi2003/Transfomer_keypoint.git


fatal: destination path 'Transfomer_keypoint' already exists and is not an empty directory.


In [6]:
data_dir = '/content/Transfomer_keypoint/MediaPipe/treino'

keypoints_list = []
labels_list = []

for class_folder in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_folder)

    for file_name in os.listdir(class_path):
        if file_name.endswith('.npy'):
            file_path = os.path.join(class_path, file_name)

            keypoints = np.load(file_path, allow_pickle=True)
            keypoints_list.append(keypoints)

            labels_list.append(int(class_folder))

keypoints_array = np.array(keypoints_list)
labels_array = np.array(labels_list)

In [7]:
keypoints_list[0][0].shape


(33, 2)

In [8]:
# Convertendo para um array NumPy
keypoints_array = np.array(keypoints_list)

# Convertendo para float32
keypoints_array = keypoints_array.astype('float32')

In [9]:
labels_array
labels_array = np.expand_dims(labels_array, axis=1)

In [10]:
import os
import numpy as np

test_data_dir = '/content/Transfomer_keypoint/MediaPipe/teste'

test_keypoints_list = []
test_labels_list = []

for class_folder in os.listdir(test_data_dir):
    class_path = os.path.join(test_data_dir, class_folder)

    for file_name in os.listdir(class_path):
        if file_name.endswith('.npy'):
            file_path = os.path.join(class_path, file_name)

            keypoints = np.load(file_path, allow_pickle=True)
            test_keypoints_list.append(keypoints)

            test_labels_list.append(int(class_folder))

test_keypoints_array = np.array(test_keypoints_list)
test_keypoints_array = test_keypoints_array.astype('float32')
test_labels_array = np.array(test_labels_list)



In [11]:
print(test_labels_array)
test_labels_array = np.expand_dims(test_labels_array, axis=1)

[0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras import callbacks

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def build(self, input_shape):
        self.position_embeddings.build(input_shape)

    def call(self, inputs):
        # Os inputs têm a forma: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        #(None, 900, 33,2)
        positions = tf.range(start=0, limit=length, delta=1)

        embedded_positions = self.position_embeddings(positions)
        embedded_positions = tf.expand_dims(embedded_positions, axis=-1)
        return inputs + embedded_positions

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(2, activation=keras.activations.gelu),
                layers.Dense(2),
                #layers.Dense(embed_dim)
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)



def get_compiled_model(shape):
    sequence_length = 900
    embed_dim = 33
    dense_dim = 4
    num_heads = 1
    classes = 2

    inputs = keras.Input(shape=shape)
    x = PositionalEmbedding(sequence_length, embed_dim)(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
    x = layers.GlobalMaxPooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    return model

shape_of_train_data = (900, 33, 2)
model = get_compiled_model(shape_of_train_data)

early_stop_callback = callbacks.EarlyStopping(monitor='val_accuracy', patience=40)
loss = tf.keras.losses.BinaryCrossentropy()
LR= 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)

metrics = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'),
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
]

model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=metrics,
)

# Treinar o modelo usando seus dados de treino e rótulos
model.fit(keypoints_array, labels_array, epochs=200, batch_size=1, validation_data = (test_keypoints_array, test_labels_array), callbacks=[early_stop_callback])

Epoch 1/200
22/22 [==============================] - 13s 215ms/step - loss: 0.8052 - tp: 16.0000 - fp: 6.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - accuracy: 0.7273 - precision: 0.7273 - recall: 1.0000 - auc: 0.5000 - val_loss: 0.5414 - val_tp: 13.0000 - val_fp: 3.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.8125 - val_precision: 0.8125 - val_recall: 1.0000 - val_auc: 0.5000
Epoch 2/200
22/22 [==============================] - 4s 184ms/step - loss: 0.8383 - tp: 16.0000 - fp: 6.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - accuracy: 0.7273 - precision: 0.7273 - recall: 1.0000 - auc: 0.5000 - val_loss: 0.5252 - val_tp: 13.0000 - val_fp: 3.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.8125 - val_precision: 0.8125 - val_recall: 1.0000 - val_auc: 0.5000
Epoch 3/200
22/22 [==============================] - 4s 184ms/step - loss: 0.6294 - tp: 16.0000 - fp: 6.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - accuracy: 0.7273 - precision: 0.7273 - recall: 1.0000 - auc: 0.5

KeyboardInterrupt: 

In [ ]:
print(test_keypoints_array[0][0])

[[0.30110693 0.28500876]
 [0.3284044  0.25892445]
 [0.33987662 0.2581696 ]
 [0.35327956 0.25702545]
 [0.31394362 0.25767797]
 [0.31507438 0.25631186]
 [0.31680992 0.2547301 ]
 [0.42208448 0.25474152]
 [0.37044987 0.25222597]
 [0.34372357 0.3080961 ]
 [0.32303378 0.30725214]
 [0.5648725  0.30473626]
 [0.3992331  0.30548212]
 [0.7058653  0.36796445]
 [0.5121184  0.31337827]
 [0.546228   0.3002747 ]
 [0.4147959  0.27092838]
 [0.49980548 0.28648534]
 [0.38528594 0.2618262 ]
 [0.48702314 0.27115107]
 [0.3824029  0.25750357]
 [0.49658757 0.27914676]
 [0.3968175  0.2648318 ]
 [0.5884474  0.53089494]
 [0.437027   0.5434442 ]
 [0.7171043  0.46464723]
 [0.35452682 0.54927814]
 [0.9066562  0.5049587 ]
 [0.34171245 0.5930849 ]
 [0.92478293 0.532203  ]
 [0.3578221  0.61715406]
 [1.0206714  0.48289335]
 [0.2675277  0.5787695 ]]


In [ ]:
# Avaliar o modelo no conjunto de teste
evaluation_result = model.evaluate(test_keypoints_array, test_labels_array)
print("Loss:", evaluation_result[0])
print("Accuracy:", evaluation_result[1])


ResourceExhaustedError: Graph execution error:

Detected at node model_2/transformer_encoder_2/multi_head_attention_2/einsum/Einsum defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 6